In [2]:
import os
import re

import anndata
import numpy as np
import pandas as pd
import scanpy as sc



## Bulk RNA-seq

In [63]:
exp = pd.read_csv('../data/bulk_rna_seq/tcga.txt', sep='\t')
exp.columns = [x.replace('-', '_') for x in exp.columns]
exp['Gene'] = [x.split('.')[0] for x in exp['Gene']]
exp['Gene'] = [x.replace('-', '_') for x in exp['Gene']]
exp = exp.groupby('Gene').mean()
exp

,TCGA_OE_A75W_01A_12R_A32O_07,TCGA_2J_AABT_01A_11R_A41B_07,TCGA_IB_7886_01A_11R_2156_07,TCGA_IB_AAUU_01A_11R_A37L_07,TCGA_2J_AAB6_01A_11R_A41B_07,TCGA_LB_A8F3_01A_11R_A36G_07,TCGA_HZ_A4BH_01A_11R_A26U_07,TCGA_HV_A5A3_11A_11R_A26U_07,TCGA_IB_7646_01A_11R_2156_07,TCGA_2J_AAB9_01A_11R_A41B_07,...,TCGA_L1_A7W4_01A_12R_A36G_07,TCGA_IB_A5SO_01A_11R_A32O_07,TCGA_IB_AAUW_01A_12R_A38C_07,TCGA_2J_AABF_01A_31R_A41B_07,TCGA_H8_A6C1_01A_11R_A32O_07,TCGA_IB_AAUN_01A_12R_A38C_07,TCGA_FB_AAPU_01A_31R_A41B_07,TCGA_HZ_8001_01A_11R_2204_07,TCGA_IB_7651_01A_11R_2156_07,TCGA_3A_A9IJ_01A_11R_A39D_07
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.41,0.11,0.07,0.12,0.17,0.13,0.28,0.26,0.06,0.17,...,0.68,0.42,0.36,0.36,0.54,0.14,0.03,0.23,0.28,2.05
A1BG_AS1,1.51,1.10,0.73,0.84,1.19,0.44,1.55,1.61,0.44,1.56,...,1.79,1.43,1.70,2.00,1.38,0.82,0.27,1.50,1.60,2.58
A1CF,3.17,5.20,17.23,12.65,1.99,12.87,10.41,4.12,0.50,4.00,...,0.16,1.94,19.68,21.56,3.74,2.65,17.77,5.76,2.35,36.73
A2M,464.50,1177.62,791.41,531.79,370.69,799.84,970.75,1378.72,746.49,1027.80,...,394.49,1091.17,1034.47,919.86,607.21,572.43,271.54,1250.65,1916.74,311.04
A2ML1,0.06,0.08,0.25,0.37,83.68,0.28,0.28,0.10,8.63,29.29,...,13.45,2.05,0.25,1.13,1.09,4.90,0.17,23.28,0.57,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.08,0.03,0.00,0.08,0.05,0.00,0.00,0.05,0.06,0.04,...,1.75,0.05,0.04,0.10,0.07,0.07,0.02,0.10,0.05,0.49
ZYG11B,13.00,50.75,36.75,31.48,31.99,13.16,32.73,30.98,31.74,28.16,...,34.46,27.88,41.68,35.72,29.35,25.08,28.01,22.89,42.44,50.54
ZYX,354.23,557.21,246.15,430.63,507.73,262.34,314.66,438.94,154.36,345.79,...,136.42,223.85,206.28,301.77,163.73,215.82,229.96,365.41,182.42,63.77


In [64]:
# exclude bad samples
freeze = pd.read_csv('../data/bulk_rna_seq/tcga/tcga_2017_samples.txt', sep='\t')
freeze

,Tumor Sample ID,Batch,Grade,Initial Slide Tumor Cellularity,Pathologist Reviewed Tumor Cellularity,ABSOLUTE Purity,Ploidy,Purity Class (high or low),DNA methylation leukocyte percent estimate,DNA hypermethylation mode purity,...,Radiation therapy,Residual tumor,History of chronic pancreatitis,Age at initial pathologic diagnosis,Date of initial pathologic diagnosis,Gender,Ethnicity,Race,Number of pack years smoked,Year of tobacco smoking onset
0,TCGA-2J-AAB1-01A,Batch 424,Grade 2 - Moderately differentiated (50% to 95...,40,7,0.38,1.9,high,0.20,0.50,...,no,r0,no,65,2012.0,male,NaN,white,25.0,1962.0
1,TCGA-2J-AAB4-01A,Batch 424,Grade 2 - Moderately differentiated (50% to 95...,40,4,0.30,2.0,low,0.20,0.32,...,no,r0,yes,48,2012.0,male,NaN,white,NaN,NaN
2,TCGA-2J-AAB6-01A,Batch 424,Grade 3 - Poorly differentiated (49% or less o...,60,25,0.54,2.4,high,0.12,0.54,...,no,r0,no,75,2012.0,male,NaN,white,NaN,NaN
3,TCGA-2J-AAB8-01A,Batch 424,Grade 2 - Moderately differentiated (50% to 95...,70,30,0.27,1.8,low,0.42,0.36,...,NaN,r1,no,71,2012.0,male,NaN,white,NaN,NaN
4,TCGA-2J-AAB9-01A,Batch 424,Grade 2 - Moderately differentiated (50% to 95...,70,9,NaN,2.0,low,0.30,0.24,...,yes,r0,no,70,2012.0,female,NaN,white,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,TCGA-XN-A8T5-01A,Batch 369,Grade 3 - Poorly differentiated (49% or less o...,100,2,0.09,2.0,low,0.59,0.22,...,yes,r1,no,53,2012.0,female,hispanic or latino,white,15.2,1974.0
146,TCGA-YB-A89D-01A,Batch 369,Grade 2 - Moderately differentiated (50% to 95...,10,18,0.24,3.3,low,0.25,0.42,...,yes,r0,no,59,2013.0,male,not hispanic or latino,white,NaN,NaN
147,TCGA-YH-A8SY-01A,Batch 390,Grade 2 - Moderately differentiated (50% to 95...,50,20,0.22,3.4,low,0.22,0.41,...,yes,NaN,no,73,2011.0,female,not hispanic or latino,white,NaN,NaN
148,TCGA-YY-A8LH-01A,Batch 369,Grade 3 - Poorly differentiated (49% or less o...,40,19,0.55,3.7,high,0.09,0.54,...,yes,r0,no,61,2009.0,female,not hispanic or latino,white,46.0,1963.0


In [65]:
list(freeze['Tumor Sample ID'])

['TCGA-2J-AAB1-01A',
 'TCGA-2J-AAB4-01A',
 'TCGA-2J-AAB6-01A',
 'TCGA-2J-AAB8-01A',
 'TCGA-2J-AAB9-01A',
 'TCGA-2J-AABA-01A',
 'TCGA-2J-AABE-01A',
 'TCGA-2J-AABF-01A',
 'TCGA-2J-AABH-01A',
 'TCGA-2J-AABI-01A',
 'TCGA-2J-AABK-01A',
 'TCGA-2J-AABO-01A',
 'TCGA-2J-AABR-01A',
 'TCGA-2J-AABT-01A',
 'TCGA-2J-AABU-01A',
 'TCGA-2J-AABV-01A',
 'TCGA-2L-AAQA-01A',
 'TCGA-2L-AAQE-01A',
 'TCGA-2L-AAQI-01A',
 'TCGA-2L-AAQJ-01A',
 'TCGA-2L-AAQL-01A',
 'TCGA-3A-A9I5-01A',
 'TCGA-3A-A9I7-01A',
 'TCGA-3A-A9I9-01A',
 'TCGA-3A-A9IB-01A',
 'TCGA-3A-A9IC-01A',
 'TCGA-3A-A9IH-01A',
 'TCGA-3A-A9IU-01A',
 'TCGA-3A-A9IX-01A',
 'TCGA-3A-A9IZ-01A',
 'TCGA-3A-A9J0-01A',
 'TCGA-3E-AAAY-01A',
 'TCGA-3E-AAAZ-01A',
 'TCGA-F2-6879-01A',
 'TCGA-F2-A44G-01A',
 'TCGA-F2-A44H-01A',
 'TCGA-F2-A7TX-01A',
 'TCGA-F2-A8YN-01A',
 'TCGA-FB-A4P5-01A',
 'TCGA-FB-A4P6-01A',
 'TCGA-FB-A545-01A',
 'TCGA-FB-A5VM-01A',
 'TCGA-FB-A78T-01A',
 'TCGA-FB-AAPQ-01A',
 'TCGA-FB-AAPS-01A',
 'TCGA-FB-AAPU-01A',
 'TCGA-FB-AAPY-01A',
 'TCGA-FB-AAP

In [66]:
'TCGA-XD-AAUL-01A' in set(freeze['Tumor Sample ID'])

True

In [67]:
[c[:16].replace('_', '-') for c in exp.columns]

['TCGA-OE-A75W-01A',
 'TCGA-2J-AABT-01A',
 'TCGA-IB-7886-01A',
 'TCGA-IB-AAUU-01A',
 'TCGA-2J-AAB6-01A',
 'TCGA-LB-A8F3-01A',
 'TCGA-HZ-A4BH-01A',
 'TCGA-HV-A5A3-11A',
 'TCGA-IB-7646-01A',
 'TCGA-2J-AAB9-01A',
 'TCGA-2J-AAB4-01A',
 'TCGA-3A-A9IX-01A',
 'TCGA-HZ-7918-01A',
 'TCGA-IB-A5SS-01A',
 'TCGA-2J-AABO-01A',
 'TCGA-2L-AAQL-01A',
 'TCGA-IB-8127-01A',
 'TCGA-HZ-A8P1-01A',
 'TCGA-FB-AAPY-01A',
 'TCGA-US-A774-01A',
 'TCGA-IB-7654-01A',
 'TCGA-XN-A8T5-01A',
 'TCGA-FB-AAPP-01A',
 'TCGA-IB-7891-01A',
 'TCGA-IB-A7LX-01A',
 'TCGA-FB-A78T-01A',
 'TCGA-XD-AAUL-01A',
 'TCGA-HZ-8002-01A',
 'TCGA-LB-A7SX-01A',
 'TCGA-HZ-8519-01A',
 'TCGA-2J-AABU-01A',
 'TCGA-2J-AABV-01A',
 'TCGA-FB-AAQ0-01A',
 'TCGA-2L-AAQJ-01A',
 'TCGA-3A-A9IU-01A',
 'TCGA-HZ-A49G-01A',
 'TCGA-HV-AA8X-01A',
 'TCGA-US-A77G-01A',
 'TCGA-HZ-8317-01A',
 'TCGA-Q3-A5QY-01A',
 'TCGA-3A-A9IR-01A',
 'TCGA-YB-A89D-11A',
 'TCGA-HZ-A49I-01A',
 'TCGA-HZ-8636-01A',
 'TCGA-YH-A8SY-01A',
 'TCGA-FB-AAQ3-01A',
 'TCGA-IB-AAUR-01A',
 'TCGA-HV-A5A

In [68]:
exp = exp[[c for c in exp.columns if c[:16].replace('_', '-') in set(freeze['Tumor Sample ID'])]]
exp


,TCGA_OE_A75W_01A_12R_A32O_07,TCGA_2J_AABT_01A_11R_A41B_07,TCGA_IB_7886_01A_11R_2156_07,TCGA_IB_AAUU_01A_11R_A37L_07,TCGA_2J_AAB6_01A_11R_A41B_07,TCGA_LB_A8F3_01A_11R_A36G_07,TCGA_HZ_A4BH_01A_11R_A26U_07,TCGA_IB_7646_01A_11R_2156_07,TCGA_2J_AAB9_01A_11R_A41B_07,TCGA_2J_AAB4_01A_12R_A41B_07,...,TCGA_3A_A9I7_01A_21R_A38C_07,TCGA_IB_7890_01A_12R_2204_07,TCGA_2J_AABH_01A_21R_A41B_07,TCGA_IB_AAUQ_01A_22R_A41I_07,TCGA_IB_A5SO_01A_11R_A32O_07,TCGA_2J_AABF_01A_31R_A41B_07,TCGA_IB_AAUN_01A_12R_A38C_07,TCGA_FB_AAPU_01A_31R_A41B_07,TCGA_HZ_8001_01A_11R_2204_07,TCGA_IB_7651_01A_11R_2156_07
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.41,0.11,0.07,0.12,0.17,0.13,0.28,0.06,0.17,0.27,...,0.23,0.35,0.24,0.48,0.42,0.36,0.14,0.03,0.23,0.28
A1BG_AS1,1.51,1.10,0.73,0.84,1.19,0.44,1.55,0.44,1.56,0.85,...,0.99,1.24,0.82,1.47,1.43,2.00,0.82,0.27,1.50,1.60
A1CF,3.17,5.20,17.23,12.65,1.99,12.87,10.41,0.50,4.00,6.65,...,24.60,3.93,6.31,12.42,1.94,21.56,2.65,17.77,5.76,2.35
A2M,464.50,1177.62,791.41,531.79,370.69,799.84,970.75,746.49,1027.80,749.02,...,661.84,932.46,620.86,812.87,1091.17,919.86,572.43,271.54,1250.65,1916.74
A2ML1,0.06,0.08,0.25,0.37,83.68,0.28,0.28,8.63,29.29,0.09,...,29.37,28.78,0.51,3.12,2.05,1.13,4.90,0.17,23.28,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.08,0.03,0.00,0.08,0.05,0.00,0.00,0.06,0.04,0.08,...,0.14,0.07,0.05,0.11,0.05,0.10,0.07,0.02,0.10,0.05
ZYG11B,13.00,50.75,36.75,31.48,31.99,13.16,32.73,31.74,28.16,31.31,...,31.50,49.63,20.93,30.43,27.88,35.72,25.08,28.01,22.89,42.44
ZYX,354.23,557.21,246.15,430.63,507.73,262.34,314.66,154.36,345.79,269.17,...,370.51,254.49,289.25,337.93,223.85,301.77,215.82,229.96,365.41,182.42


In [69]:
exp.to_csv('../data/ecotyper/bulk/tcga.txt', sep='\t')

In [70]:
list(freeze.columns)

['Tumor Sample ID',
 'Batch',
 'Grade',
 'Initial Slide Tumor Cellularity',
 'Pathologist Reviewed Tumor Cellularity',
 'ABSOLUTE Purity',
 'Ploidy',
 'Purity Class (high or low)',
 'DNA methylation leukocyte percent estimate',
 'DNA hypermethylation mode purity',
 'mRNA Moffitt clusters (76 High Purity Samples Only)  1basal  2classical',
 'mRNA Moffitt clusters (All 150 Samples) 1basal  2classical',
 'mRNA Collisson clusters (All 150 Samples) 1classical 2exocrine 3QM',
 'mRNA Bailey Clusters (All 150 Samples) 1squamous 2immunogenic 3progenitor 4ADEX',
 'Methylation Clusters (76 High Purity Samples Only)',
 'Methylation Clusters (All 150 Samples)',
 'Copy Number Clusters (All 150 Samples)',
 'miRNA Clusters (All 150 Samples)',
 'lncRNA Clusters (All 150 Samples)',
 'RPPA Clusters (76 High Purity Samples Only)',
 'RPPA Clusters (All 150 Samples)',
 'KRAS Mutated (1 or 0)',
 'CDKN2A Status',
 'CDKN2A Expression',
 'Mutated SMGs',
 'Mutated PanCan Genes',
 'Mutated CGC Genes',
 'Clinical 

In [71]:
meta = freeze[['Tumor Sample ID',
               'Purity Class (high or low)',
               'mRNA Moffitt clusters (All 150 Samples) 1basal  2classical',
              'mRNA Collisson clusters (All 150 Samples) 1classical 2exocrine 3QM',
               'mRNA Bailey Clusters (All 150 Samples) 1squamous 2immunogenic 3progenitor 4ADEX',
              'Methylation Clusters (All 150 Samples)',
              'Copy Number Clusters (All 150 Samples)',
              'RPPA Clusters (All 150 Samples)',
              'KRAS Mutated (1 or 0)',
              'Follow up vital status',
              'AJCC pathologic tumor stage',
              'Radiation therapy',
              'Follow up days',
              'Censored 1=yes 0=no']]
meta.columns = ['ID', 'Purity', 'Moffit', 'Collison', 'Bailey', 'Methylation', 'Copy Number', 'RPPA', 'KRAS mutation',
               'Vital status', 'Stage', 'Radiation therapy', 'Survival time', 'Status']
meta

,ID,Purity,Moffit,Collison,Bailey,Methylation,Copy Number,RPPA,KRAS mutation,Vital status,Stage,Radiation therapy,Survival time,Status
0,TCGA-2J-AAB1-01A,high,2,1,3,1.0,Low,NaN,1,Dead,stage iib_n1,no,66.0,0.0
1,TCGA-2J-AAB4-01A,low,2,1,3,NaN,Low,NaN,1,Alive,stage iib_n1,no,729.0,1.0
2,TCGA-2J-AAB6-01A,high,1,3,1,1.0,Low,NaN,1,Dead,stage ia.ib.iia_n0,no,293.0,0.0
3,TCGA-2J-AAB8-01A,low,2,1,3,1.0,High,NaN,1,Alive,stage iib_n1,NaN,80.0,1.0
4,TCGA-2J-AAB9-01A,low,1,2,4,NaN,Low,NaN,1,Dead,stage iib_n1,yes,627.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,TCGA-XN-A8T5-01A,low,1,3,2,NaN,non-Aneuploid,3.0,1,Alive,stage ia.ib.iia_n0,yes,720.0,1.0
146,TCGA-YB-A89D-01A,low,2,1,3,2.0,Low,2.0,1,Alive,stage iib_n1,yes,350.0,1.0
147,TCGA-YH-A8SY-01A,low,1,3,1,2.0,High,1.0,1,Alive,stage iii,yes,388.0,1.0
148,TCGA-YY-A8LH-01A,high,2,1,3,1.0,Low,3.0,1,Alive,stage iib_n1,yes,2016.0,1.0


In [72]:
m = {1:'basal', 2:'classical'}
meta['Moffit'] = [m[int(x)] for x in meta['Moffit']]
m = {1:'classical', 2:'exocrine', 3:'quasi-mesanchymal'}
meta['Collison'] = [m[int(x)] for x in meta['Collison']]
m = {1:'squamous-like', 2:'immunogenic', 3:'pancreatic progenitor', 4:'ADEX'}
meta['Bailey'] = [m[int(x)] for x in meta['Bailey']]
meta['Methylation'] = [int(x) if not pd.isnull(x) else 'unassigned' for x in meta['Methylation']]
meta['Copy Number'] = [x if not pd.isnull(x) else 'unassigned' for x in meta['Copy Number']]
meta['RPPA'] = [int(x) if not pd.isnull(x) else 'unassigned' for x in meta['RPPA']]
meta['KRAS mutation'] = ['yes' if x==1 else 'no' for x in meta['KRAS mutation']]
meta['Stage'] = [re.sub(r'^(stage [^\._]).*$', r'\1', x) if not pd.isnull(x) else 'unknown'
                 for x in meta['Stage']]
meta['Radiation therapy'] = [x if not pd.isnull(x) else 'unassigned' for x in meta['Radiation therapy']]
meta['Survival time'] = [int(x / 30) + 1 if not pd.isnull(x) else np.nan for x in meta['Survival time']]
meta['Status'] = [int(x) + 1 if not pd.isnull(x) else np.nan for x in meta['Status']]
meta

/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_pdac_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_pdac_v2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_pdac_v2/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to

,ID,Purity,Moffit,Collison,Bailey,Methylation,Copy Number,RPPA,KRAS mutation,Vital status,Stage,Radiation therapy,Survival time,Status
0,TCGA-2J-AAB1-01A,high,classical,classical,pancreatic progenitor,1,Low,unassigned,yes,Dead,stage i,no,3.0,1.0
1,TCGA-2J-AAB4-01A,low,classical,classical,pancreatic progenitor,unassigned,Low,unassigned,yes,Alive,stage i,no,25.0,2.0
2,TCGA-2J-AAB6-01A,high,basal,quasi-mesanchymal,squamous-like,1,Low,unassigned,yes,Dead,stage i,no,10.0,1.0
3,TCGA-2J-AAB8-01A,low,classical,classical,pancreatic progenitor,1,High,unassigned,yes,Alive,stage i,unassigned,3.0,2.0
4,TCGA-2J-AAB9-01A,low,basal,exocrine,ADEX,unassigned,Low,unassigned,yes,Dead,stage i,yes,21.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,TCGA-XN-A8T5-01A,low,basal,quasi-mesanchymal,immunogenic,unassigned,non-Aneuploid,3,yes,Alive,stage i,yes,25.0,2.0
146,TCGA-YB-A89D-01A,low,classical,classical,pancreatic progenitor,2,Low,2,yes,Alive,stage i,yes,12.0,2.0
147,TCGA-YH-A8SY-01A,low,basal,quasi-mesanchymal,squamous-like,2,High,1,yes,Alive,stage i,yes,13.0,2.0
148,TCGA-YY-A8LH-01A,high,classical,classical,pancreatic progenitor,1,Low,3,yes,Alive,stage i,yes,68.0,2.0


In [73]:
np.count_nonzero(pd.isnull(meta.values))

8

In [74]:
d = {}
for s1 in exp.columns:
    for s2 in meta['ID']:
        if s2.replace('-', '_') in s1:
            d[s2] = s1
len(d), d

(149,
 {'TCGA-OE-A75W-01A': 'TCGA_OE_A75W_01A_12R_A32O_07',
  'TCGA-2J-AABT-01A': 'TCGA_2J_AABT_01A_11R_A41B_07',
  'TCGA-IB-7886-01A': 'TCGA_IB_7886_01A_11R_2156_07',
  'TCGA-IB-AAUU-01A': 'TCGA_IB_AAUU_01A_11R_A37L_07',
  'TCGA-2J-AAB6-01A': 'TCGA_2J_AAB6_01A_11R_A41B_07',
  'TCGA-LB-A8F3-01A': 'TCGA_LB_A8F3_01A_11R_A36G_07',
  'TCGA-HZ-A4BH-01A': 'TCGA_HZ_A4BH_01A_11R_A26U_07',
  'TCGA-IB-7646-01A': 'TCGA_IB_7646_01A_11R_2156_07',
  'TCGA-2J-AAB9-01A': 'TCGA_2J_AAB9_01A_11R_A41B_07',
  'TCGA-2J-AAB4-01A': 'TCGA_2J_AAB4_01A_12R_A41B_07',
  'TCGA-3A-A9IX-01A': 'TCGA_3A_A9IX_01A_11R_A41B_07',
  'TCGA-IB-A5SS-01A': 'TCGA_IB_A5SS_01A_11R_A32O_07',
  'TCGA-2J-AABO-01A': 'TCGA_2J_AABO_01A_21R_A41B_07',
  'TCGA-2L-AAQL-01A': 'TCGA_2L_AAQL_01A_11R_A38C_07',
  'TCGA-IB-8127-01A': 'TCGA_IB_8127_01A_11R_2404_07',
  'TCGA-HZ-A8P1-01A': 'TCGA_HZ_A8P1_01A_11R_A37L_07',
  'TCGA-FB-AAPY-01A': 'TCGA_FB_AAPY_01A_11R_A41B_07',
  'TCGA-US-A774-01A': 'TCGA_US_A774_01A_21R_A32O_07',
  'TCGA-XN-A8T5-01A': 

In [75]:
meta = meta[[True if x in d else False for x in meta['ID']]]
meta['ID'] = [d[x] for x in meta['ID']]
meta

/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_pdac_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,Purity,Moffit,Collison,Bailey,Methylation,Copy Number,RPPA,KRAS mutation,Vital status,Stage,Radiation therapy,Survival time,Status
0,TCGA_2J_AAB1_01A_11R_A41B_07,high,classical,classical,pancreatic progenitor,1,Low,unassigned,yes,Dead,stage i,no,3.0,1.0
1,TCGA_2J_AAB4_01A_12R_A41B_07,low,classical,classical,pancreatic progenitor,unassigned,Low,unassigned,yes,Alive,stage i,no,25.0,2.0
2,TCGA_2J_AAB6_01A_11R_A41B_07,high,basal,quasi-mesanchymal,squamous-like,1,Low,unassigned,yes,Dead,stage i,no,10.0,1.0
3,TCGA_2J_AAB8_01A_12R_A41B_07,low,classical,classical,pancreatic progenitor,1,High,unassigned,yes,Alive,stage i,unassigned,3.0,2.0
4,TCGA_2J_AAB9_01A_11R_A41B_07,low,basal,exocrine,ADEX,unassigned,Low,unassigned,yes,Dead,stage i,yes,21.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,TCGA_XN_A8T5_01A_12R_A36G_07,low,basal,quasi-mesanchymal,immunogenic,unassigned,non-Aneuploid,3,yes,Alive,stage i,yes,25.0,2.0
146,TCGA_YB_A89D_01A_12R_A36G_07,low,classical,classical,pancreatic progenitor,2,Low,2,yes,Alive,stage i,yes,12.0,2.0
147,TCGA_YH_A8SY_01A_11R_A37L_07,low,basal,quasi-mesanchymal,squamous-like,2,High,1,yes,Alive,stage i,yes,13.0,2.0
148,TCGA_YY_A8LH_01A_11R_A36G_07,high,classical,classical,pancreatic progenitor,1,Low,3,yes,Alive,stage i,yes,68.0,2.0


In [76]:
meta.columns = [x.replace(' ', '.') for x in meta.columns]

In [77]:
meta.to_csv('../data/ecotyper/bulk/tcga_annotations.txt', sep='\t', index=False)

In [78]:
list(meta.columns)

['ID',
 'Purity',
 'Moffit',
 'Collison',
 'Bailey',
 'Methylation',
 'Copy.Number',
 'RPPA',
 'KRAS.mutation',
 'Vital.status',
 'Stage',
 'Radiation.therapy',
 'Survival.time',
 'Status']

In [124]:
exp = pd.read_csv('../data/bulk_rna_seq/bailey.txt', sep='\t')
exp.columns = [x.replace('-', '_') for x in exp.columns]
exp['Gene'] = [x.split('.')[0] for x in exp['Gene']]
exp['Gene'] = [x.replace('-', '_') for x in exp['Gene']]
exp = exp.groupby('Gene').mean()
exp

,SA518851,SA528771,SA528766,SA528763,SA518817,SA518765,SA518854,SA518806,SA528767,SA518750,...,SA412299,SA411841,SA411797,SA411769,SA411833,SA411430,SA411406,SA411923,SA411305,SA411682
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,1.873957,1.781443,1.851646,1.125449,1.267181,0.842934,0.963219,0.967079,2.390214,1.163613,...,3.788089,0.131573,1.538798,1.456039,0.883073,1.269928,1.874871,2.855517,1.305073,0.216444
A1BG_AS1,1.117166,0.950103,0.925823,2.077753,1.267181,0.991687,0.963219,0.511983,2.390214,0.775742,...,1.638818,0.000000,1.471894,2.053387,1.527478,0.893653,2.879267,3.824354,1.305073,0.270556
A1CF,0.000000,3.206598,3.240381,6.406403,3.225552,1.661076,1.083622,0.113774,8.795989,6.119744,...,2.471661,0.000000,7.292564,29.643450,2.577619,6.584809,2.644908,2.498578,1.334733,0.108222
A2M,246.677580,140.021440,97.674330,318.155850,461.791500,763.251950,921.399540,228.799410,427.083530,887.750800,...,236.043580,0.000000,245.070300,613.029500,689.298340,647.710000,960.637200,647.641540,359.784800,2.326777
A2ML1,192.224730,2.018969,15.276081,0.779157,0.153598,62.129196,28.214298,41.299946,0.669260,0.603355,...,0.026866,34.077328,0.970112,0.821355,1.121742,0.047034,61.569440,6.883837,63.918896,16.179220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,27.136335,50.236700,54.160652,31.339434,28.031582,32.899220,41.619100,55.407913,40.538036,41.631500,...,39.949560,11.622254,40.142555,34.198235,29.356220,33.770664,38.100063,43.903580,37.906425,14.988775
ZYG11A,0.396414,4.275464,4.166204,2.337472,0.844787,0.619804,1.163890,1.080852,6.692601,1.077420,...,0.268659,0.043858,0.802851,0.709352,0.668272,0.282206,0.401758,1.376767,0.474572,4.815888
ZYG11B,76.435814,48.336490,51.846092,67.267235,75.685270,74.004650,77.338480,73.497970,64.248960,121.791530,...,53.543690,41.664684,56.132675,64.849710,78.307120,54.795017,59.594128,84.084790,119.147194,69.586876


In [5]:
exp.to_csv('../data/ecotyper/bulk/bailey.txt', sep='\t')

In [6]:
exp = pd.read_csv('../data/bulk_rna_seq/kirby.txt', sep='\t')
exp.columns = [x.replace('-', '_') for x in exp.columns]
exp['Gene'] = [x.split('.')[0] for x in exp['Gene']]
exp['Gene'] = [x.replace('-', '_') for x in exp['Gene']]
exp = exp.groupby('Gene').mean()
exp

,T_07_07_A082a,T_06_01_A033a,T_06_11_A168a,T_06_04_A296a,T_07_11_A090a,T_06_06_A349a,T_11_03_A138a,T_03_11_A244a,T_10_11_A059a,T_04_02_A274c,...,T_10_11_A100a,T_10_11_A128a,T_10_11_A130b,T_10_12_A057a,T_11_02_A148a,T_11_02_A171a_2,T_11_04_A026b,T_11_05_A018b,T_11_06_A011a,T_11_06_A104a
Gene,,,,,,,,,,,,,,,,,,,,,
5S,0.07,0.28,0.00,0.15,0.17,0.37,0.15,0.29,0.03,0.05,...,0.20,0.03,0.08,0.26,0.20,1.25,0.16,0.19,0.21,0.13
7SK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A1BG,7.77,2.90,6.55,12.48,6.99,4.48,6.60,1.27,7.36,5.37,...,8.52,7.53,6.89,4.22,3.65,4.08,10.19,5.22,14.34,5.41
A1CF,4.20,5.76,6.60,1.53,2.00,0.50,2.82,3.79,0.16,0.03,...,3.13,3.58,0.38,0.33,4.74,0.45,5.84,7.18,1.22,0.76
A2LD1,2.24,2.00,1.06,1.58,1.78,0.91,1.54,2.62,2.40,0.77,...,1.43,0.98,0.25,2.92,0.98,1.31,1.80,2.31,1.82,1.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
snoZ5,0.78,0.60,0.50,0.54,0.41,0.54,0.80,0.51,0.19,0.52,...,0.49,0.64,0.34,1.21,0.33,0.32,0.99,0.47,0.53,0.60
snoZ6,0.00,0.01,0.03,0.01,0.01,0.06,0.05,0.01,0.04,0.02,...,0.03,0.01,0.02,0.00,0.02,0.00,0.01,0.08,0.01,0.00
snosnR60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [7]:
exp.to_csv('../data/ecotyper/bulk/kirby.txt', sep='\t')

In [8]:
exp = pd.read_csv('../data/bulk_rna_seq/moffit.txt', sep='\t')
exp.columns = [x.replace('-', '_') for x in exp.columns]
exp['Gene'] = [x.split('.')[0] for x in exp['Gene']]
exp['Gene'] = [x.replace('-', '_') for x in exp['Gene']]
exp = exp.groupby('Gene').mean()
exp = pd.DataFrame(data=np.exp2(exp.values), index=exp.index, columns=exp.columns)
exp

,GSM1844105,GSM1844106,GSM1844107,GSM1844108,GSM1844109,GSM1844110,GSM1844111,GSM1844112,GSM1844113,GSM1844114,...,GSM1844240,GSM1844241,GSM1844242,GSM1844243,GSM1844244,GSM1844245,GSM1844246,GSM1844247,GSM1844248,GSM1844249
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,6.307201,11.712686,5.617780,6.117745,7.285359,7.917253,11.623720,6.241964,12.295001,6.806910,...,6.395247,7.305586,10.042176,11.639845,7.679434,12.277969,9.586472,15.412190,11.337259,13.632706
A1CF,2.812786,4.304960,4.236873,11.173428,18.635737,18.791392,4.900741,7.537050,4.161201,2.284692,...,2.308571,2.203810,4.843332,2.542063,5.329499,4.272262,3.427128,35.976926,2.696335,5.649018
A2BP1,2.620787,1.831467,1.898684,1.977942,1.750783,1.674813,1.700548,1.922522,2.331082,1.425038,...,2.838247,1.836552,2.770219,2.840215,1.678299,2.130216,2.323017,2.743467,3.206501,2.142062
A2LD1,5.629474,14.866571,15.021948,8.202138,28.640802,28.207365,6.241964,11.408206,18.252219,8.391915,...,10.973866,9.037978,10.396312,17.691690,12.025292,14.938879,8.374482,9.593119,6.689971,45.759520
A2M,756.921565,522.034014,781.986559,1514.892809,527.124523,672.318016,1849.603093,53.817371,871.888597,2372.183667,...,1850.885584,1563.973118,1018.337453,568.099570,2367.255975,3771.702056,2951.024126,1387.238377,488.088855,366.838182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hCG_20426,13.259918,14.015973,8.760420,18.417432,26.245533,26.046182,21.917328,25.545548,20.070435,14.580820,...,12.269461,13.851785,27.020828,10.161208,9.202335,13.426394,10.996710,23.539308,13.379942,13.765642
hCG_2045899,6.220368,2.316586,3.896017,2.479415,2.838247,6.324713,5.656854,2.265768,2.733975,3.961371,...,20.850354,17.423952,4.368082,8.742222,6.543216,19.172944,14.300563,6.395247,2.519260,24.761040
hCG_21078,1968.662381,2067.970657,1316.963823,1338.127620,887.744065,1155.258509,1272.985261,1744.986307,772.290599,565.349810,...,1360.574274,1294.339173,2067.970657,1926.811868,1520.152136,1624.747711,2175.302014,1926.811868,1884.544308,1520.152136


In [9]:
exp.to_csv('../data/ecotyper/bulk/moffit.txt', sep='\t')

## scRNA-seq

In [41]:
adata = sc.read_h5ad('../data/combined/eus_only.h5ad')
adata

/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/anndata/compat/__init__.py:182: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,


AnnData object with n_obs × n_vars = 25532 × 3000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'SCT_snn_res.2', 'SCT_snn_res.1.5', 'cell_type'
    var: 'sct.detection_rate', 'sct.gmean', 'sct.variance', 'sct.residual_mean', 'sct.residual_variance', 'sct.variable'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances'

In [42]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,nCount_SCT,nFeature_SCT,SCT_snn_res.0.8,seurat_clusters,SCT_snn_res.2,SCT_snn_res.1.5,cell_type
wu01_AAACCTGCACATAACC-1,wu01,11822.0,3295,14.016241,3657.0,1363,0,6,6,8,Macrophage
wu01_AAACCTGCACCACGTG-1,wu01,469.0,293,5.970149,2476.0,506,0,31,31,29,Macrophage
wu01_AAACCTGTCTAACTTC-1,wu01,482.0,243,0.000000,2703.0,450,8,2,2,5,Malignant
wu01_AAACGGGCAAGGCTCC-1,wu01,1516.0,717,2.704485,3168.0,726,2,16,16,0,Macrophage
wu01_AAACGGGTCATTCACT-1,wu01,876.0,453,7.648402,2715.0,548,0,6,6,8,Macrophage
...,...,...,...,...,...,...,...,...,...,...,...
wu32_TTTCCTCCAAAGCAAT-1,wu32,93004.0,9592,10.139349,4416.0,2145,25,41,41,36,Malignant
wu32_TTTCCTCTCGTTGCCT-1,wu32,36230.0,5006,2.362683,4275.0,1341,2,5,5,0,Macrophage
wu32_TTTGCGCAGTCTCCTC-1,wu32,47816.0,8027,9.896269,4349.0,1999,25,41,41,36,Malignant
wu32_TTTGGTTTCTCGCATC-1,wu32,29683.0,7059,20.577435,4289.0,2323,25,41,41,36,Malignant


In [43]:
adata.obs['sample_id'] = adata.obs['orig.ident'].to_list()
adata.obs['barcode'] = [re.sub(r'^.*_([ATCG]+..)$', r'\1', x) for x in adata.obs.index]
adata.obs['cell_ids'] = [f'{s}-{b}'.replace('-', '.').replace(' ', '.').replace('_', '.')
                         for s, b in zip(adata.obs['sample_id'], adata.obs['barcode'])]
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,nCount_SCT,nFeature_SCT,SCT_snn_res.0.8,seurat_clusters,SCT_snn_res.2,SCT_snn_res.1.5,cell_type,sample_id,barcode,cell_ids
wu01_AAACCTGCACATAACC-1,wu01,11822.0,3295,14.016241,3657.0,1363,0,6,6,8,Macrophage,wu01,AAACCTGCACATAACC-1,wu01.AAACCTGCACATAACC.1
wu01_AAACCTGCACCACGTG-1,wu01,469.0,293,5.970149,2476.0,506,0,31,31,29,Macrophage,wu01,AAACCTGCACCACGTG-1,wu01.AAACCTGCACCACGTG.1
wu01_AAACCTGTCTAACTTC-1,wu01,482.0,243,0.000000,2703.0,450,8,2,2,5,Malignant,wu01,AAACCTGTCTAACTTC-1,wu01.AAACCTGTCTAACTTC.1
wu01_AAACGGGCAAGGCTCC-1,wu01,1516.0,717,2.704485,3168.0,726,2,16,16,0,Macrophage,wu01,AAACGGGCAAGGCTCC-1,wu01.AAACGGGCAAGGCTCC.1
wu01_AAACGGGTCATTCACT-1,wu01,876.0,453,7.648402,2715.0,548,0,6,6,8,Macrophage,wu01,AAACGGGTCATTCACT-1,wu01.AAACGGGTCATTCACT.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wu32_TTTCCTCCAAAGCAAT-1,wu32,93004.0,9592,10.139349,4416.0,2145,25,41,41,36,Malignant,wu32,TTTCCTCCAAAGCAAT-1,wu32.TTTCCTCCAAAGCAAT.1
wu32_TTTCCTCTCGTTGCCT-1,wu32,36230.0,5006,2.362683,4275.0,1341,2,5,5,0,Macrophage,wu32,TTTCCTCTCGTTGCCT-1,wu32.TTTCCTCTCGTTGCCT.1
wu32_TTTGCGCAGTCTCCTC-1,wu32,47816.0,8027,9.896269,4349.0,1999,25,41,41,36,Malignant,wu32,TTTGCGCAGTCTCCTC-1,wu32.TTTGCGCAGTCTCCTC.1
wu32_TTTGGTTTCTCGCATC-1,wu32,29683.0,7059,20.577435,4289.0,2323,25,41,41,36,Malignant,wu32,TTTGGTTTCTCGCATC-1,wu32.TTTGGTTTCTCGCATC.1


In [44]:
list(adata.X[0].flatten())[:100]

[-0.3619008937311913,
 -0.3917098526753504,
 -0.42431834420426606,
 -0.04015757952457107,
 -0.3557929966585785,
 -0.1496994766498517,
 -0.5342465604176455,
 -0.14827876825046113,
 -0.5964441350234271,
 -0.2564988967477745,
 9.128828370568476,
 -0.16171477033453555,
 1.9776246468746344,
 2.5689289357700495,
 -0.06216266969374088,
 -0.08945925146779482,
 2.1198781279594443,
 -0.1919237253104856,
 -0.361931886052226,
 7.035417425646313,
 -0.06369397626293717,
 -0.6613992875885834,
 -0.6171167356415653,
 -0.28180941797470765,
 2.2832555634663745,
 -0.08924293243188422,
 -0.7838435847175725,
 7.0009940895014315,
 0.6552269888558742,
 -1.4809381919933622,
 -0.7526168280288772,
 -0.28366310094620734,
 0.6604223789649469,
 -0.22316439525687679,
 0.9479034867126386,
 -0.8147311740226685,
 -0.37136583285988917,
 -0.5187900084114849,
 -0.8961599556551902,
 5.767442321650307,
 -1.134285370945047,
 -0.20534653699062017,
 -0.5868775001538367,
 -0.2329812254223205,
 -0.8391712549124056,
 -0.738814229

In [45]:
list(adata.raw.X[0].toarray().flatten())[:100]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.3862943611198906,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0986122886681096,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [46]:
a = adata.copy()
a.obs = a.obs.set_index('cell_ids')
a.obs = a.obs[['sample_id', 'cell_type']]
a

AnnData object with n_obs × n_vars = 25532 × 3000
    obs: 'sample_id', 'cell_type'
    var: 'sct.detection_rate', 'sct.gmean', 'sct.variance', 'sct.residual_mean', 'sct.residual_variance', 'sct.variable'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances'

In [47]:
from collections import Counter
Counter(a.obs['cell_type']).most_common()

[('Macrophage', 6407),
 ('CD4.T.cell', 5389),
 ('Malignant', 5257),
 ('CD8.T.cell', 2276),
 ('Erythrocyte', 1544),
 ('NK', 1240),
 ('B.cell', 1211),
 ('Fibroblast', 838),
 ('cDC', 510),
 ('Plasma', 297),
 ('Acinar', 234),
 ('Endothelial', 199),
 ('Mast', 97),
 ('pDC', 33)]

In [48]:
a = a[a.obs['cell_type']!='pDC']
a

View of AnnData object with n_obs × n_vars = 25499 × 3000
    obs: 'sample_id', 'cell_type'
    var: 'sct.detection_rate', 'sct.gmean', 'sct.variance', 'sct.residual_mean', 'sct.residual_variance', 'sct.variable'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances'

In [49]:
exp = pd.DataFrame(data=a.raw.X.toarray(), columns=a.raw.var.index.to_list(), index=a.obs.index.to_list())
exp

,AL669831.5,FAM87B,LINC00115,SAMD11,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,AGRN,...,AC020934.1,AC005776.2,CYP4F8,AC008655.1,AC244250.1,LINC01658,AL008723.1,AL022318.1,AP000561.1,DSCAM
wu01.AAACCTGCACATAACC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu01.AAACCTGCACCACGTG.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu01.AAACCTGTCTAACTTC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu01.AAACGGGCAAGGCTCC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.098612,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu01.AAACGGGTCATTCACT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wu32.TTTCCTCCAAAGCAAT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu32.TTTCCTCTCGTTGCCT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu32.TTTGCGCAGTCTCCTC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu32.TTTGGTTTCTCGCATC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
exp = exp.transpose()
exp['gene'] = [x.split('.')[0] for x in exp.index.to_list()]
exp = exp.sort_values('gene')
exp = exp.groupby('gene').mean()
exp.index.name = 'Gene'
exp

,wu01.AAACCTGCACATAACC.1,wu01.AAACCTGCACCACGTG.1,wu01.AAACCTGTCTAACTTC.1,wu01.AAACGGGCAAGGCTCC.1,wu01.AAACGGGTCATTCACT.1,wu01.AAACGGGTCTGTGCAA.1,wu01.AAAGCAAAGCCACCTG.1,wu01.AAAGTAGAGAACTGTA.1,wu01.AAAGTAGGTCGGGTCT.1,wu01.AAATGCCTCCCAGGTG.1,...,wu32.TTGGCAAAGTGAAGAG.1,wu32.TTGGCAATCATCATTC.1,wu32.TTGTAGGTCGTTACGA.1,wu32.TTTACTGCAGGCGATA.1,wu32.TTTATGCCACCTATCC.1,wu32.TTTCCTCCAAAGCAAT.1,wu32.TTTCCTCTCGTTGCCT.1,wu32.TTTGCGCAGTCTCCTC.1,wu32.TTTGGTTTCTCGCATC.1,wu32.TTTGTCACACGGTGTC.1
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000000,0.0,0.0,0.000000,0.0,0.693147,0.0,0.0,0.000000,0.693147,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.0,0.000000,0.000000
A1BG-AS1,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
A1CF,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.693147,0.693147,0.000000,0.0,0.000000,0.693147
A2M,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.0,0.000000,0.000000
A2M-AS1,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDB,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
ZXDC,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
ZYG11B,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [51]:
meta = a.obs[['cell_type', 'sample_id']]
meta['ID'] = meta.index.to_list()
meta.columns = ['CellType', 'Sample', 'ID']
meta.index.name = 'Index'
meta

/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CellType,Sample,ID
Index,,,
wu01.AAACCTGCACATAACC.1,Macrophage,wu01,wu01.AAACCTGCACATAACC.1
wu01.AAACCTGCACCACGTG.1,Macrophage,wu01,wu01.AAACCTGCACCACGTG.1
wu01.AAACCTGTCTAACTTC.1,Malignant,wu01,wu01.AAACCTGTCTAACTTC.1
wu01.AAACGGGCAAGGCTCC.1,Macrophage,wu01,wu01.AAACGGGCAAGGCTCC.1
wu01.AAACGGGTCATTCACT.1,Macrophage,wu01,wu01.AAACGGGTCATTCACT.1
...,...,...,...
wu32.TTTCCTCCAAAGCAAT.1,Malignant,wu32,wu32.TTTCCTCCAAAGCAAT.1
wu32.TTTCCTCTCGTTGCCT.1,Macrophage,wu32,wu32.TTTCCTCTCGTTGCCT.1
wu32.TTTGCGCAGTCTCCTC.1,Malignant,wu32,wu32.TTTGCGCAGTCTCCTC.1


In [52]:
exp.to_csv(f'../data/ecotyper/single_cell/eusfnb_exp_12152021.txt', sep='\t')

In [53]:
meta.to_csv(f'../data/ecotyper/single_cell/eusfnb_exp_12152021_annotations.txt', sep='\t')

In [81]:
adata = sc.read_h5ad('../data/combined/ecotyper_sc_start_08232021.h5ad')
adata

AnnData object with n_obs × n_vars = 151336 × 30840
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'CELL', 'CONDITION', 'Patient', 'Type', 'Cell_type', 'integrated_snn_res.1.25', 'seurat_clusters', 'integrated_snn_res.2', 'integrated_snn_res.3', 'integrated_snn_res.1', 'integrated_snn_res.0.5', 'active.ident', 'cell_type', 'dataset', 'sample_id', 'cell_type_level_2', 'cell_type_level_3', 'tumor_original_seurat', 'malignant_exclude', 'final_cell_types'
    var: 'is_highly_variable'
    uns: 'cell_type_colors', 'cell_type_level_2_colors', 'cell_type_level_3_colors', 'dataset_colors', 'rank_genes_groups'
    obsm: 'X_umap'
    layers: 'counts'

In [82]:
# make sure there are no possible issues with ids
# ecotyper is finicky about that
adata.var

,is_highly_variable
AL669831.5,True
FAM87B,True
LINC00115,True
SAMD11,True
NOC2L,True
...,...
RP11-108P20.4,True
GS1-122H1.2,True
SLC26A4-AS1,True
RP11-403B2.6,True


In [83]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,CELL,CONDITION,Patient,Type,Cell_type,...,integrated_snn_res.0.5,active.ident,cell_type,dataset,sample_id,cell_type_level_2,cell_type_level_3,tumor_original_seurat,malignant_exclude,final_cell_types
wu01_AAACCTGCACATAACC-1,wu01,11822.0,3295,3377.0,1314,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,14,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte
wu01_AAACCTGCACCACGTG-1,wu01,469.0,293,2235.0,450,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,14,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte
wu01_AAACCTGTCTAACTTC-1,wu01,482.0,243,2436.0,400,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,0,Malignant,Malignant,In-house: EUS-FNB,wu01,Malignant_0,Malignant_0,Malignant_0,False,Malignant - Classical Low
wu01_AAACGGGCAAGGCTCC-1,wu01,1516.0,717,2944.0,717,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,2,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte
wu01_AAACGGGTCATTCACT-1,wu01,876.0,453,2491.0,524,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,14,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCAAGTTGTCGT-1,G9903,6484.0,1875,9150.0,1886,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,1,CAF,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCACAACTTGAC-1,G9903,8367.0,2286,9400.0,2286,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,1,CAF,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCACATATGGTC-1,G9903,11476.0,3055,10387.0,3055,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,16,immune,Immune,Chan-Seng-Yue,G9903,immune,U1,immune,False,U1
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCATCCGAACGC-1,G9903,20314.0,3975,10684.0,3464,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,0,Malignant,Malignant,Chan-Seng-Yue,G9903,Malignant_0,Malignant_0,Malignant_0,False,Malignant - Classical Low


In [84]:
adata.obs['barcode'] = [re.sub(r'^.*_([ATCG]+..)$', r'\1', x) for x in adata.obs.index]
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,CELL,CONDITION,Patient,Type,Cell_type,...,active.ident,cell_type,dataset,sample_id,cell_type_level_2,cell_type_level_3,tumor_original_seurat,malignant_exclude,final_cell_types,barcode
wu01_AAACCTGCACATAACC-1,wu01,11822.0,3295,3377.0,1314,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte,AAACCTGCACATAACC-1
wu01_AAACCTGCACCACGTG-1,wu01,469.0,293,2235.0,450,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte,AAACCTGCACCACGTG-1
wu01_AAACCTGTCTAACTTC-1,wu01,482.0,243,2436.0,400,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Malignant,Malignant,In-house: EUS-FNB,wu01,Malignant_0,Malignant_0,Malignant_0,False,Malignant - Classical Low,AAACCTGTCTAACTTC-1
wu01_AAACGGGCAAGGCTCC-1,wu01,1516.0,717,2944.0,717,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte,AAACGGGCAAGGCTCC-1
wu01_AAACGGGTCATTCACT-1,wu01,876.0,453,2491.0,524,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte,AAACGGGTCATTCACT-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCAAGTTGTCGT-1,G9903,6484.0,1875,9150.0,1886,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,CAF,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF,TTTGTCAAGTTGTCGT-1
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCACAACTTGAC-1,G9903,8367.0,2286,9400.0,2286,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,CAF,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF,TTTGTCACAACTTGAC-1
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCACATATGGTC-1,G9903,11476.0,3055,10387.0,3055,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,immune,Immune,Chan-Seng-Yue,G9903,immune,U1,immune,False,U1,TTTGTCACATATGGTC-1
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCATCCGAACGC-1,G9903,20314.0,3975,10684.0,3464,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Malignant,Malignant,Chan-Seng-Yue,G9903,Malignant_0,Malignant_0,Malignant_0,False,Malignant - Classical Low,TTTGTCATCCGAACGC-1


In [85]:
adata.obs['cell_ids'] = [f'{s}-{b}'.replace('-', '.').replace(' ', '.').replace('_', '.')
                         for s, b in zip(adata.obs['sample_id'], adata.obs['barcode'])]
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,CELL,CONDITION,Patient,Type,Cell_type,...,cell_type,dataset,sample_id,cell_type_level_2,cell_type_level_3,tumor_original_seurat,malignant_exclude,final_cell_types,barcode,cell_ids
wu01_AAACCTGCACATAACC-1,wu01,11822.0,3295,3377.0,1314,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte,AAACCTGCACATAACC-1,wu01.AAACCTGCACATAACC.1
wu01_AAACCTGCACCACGTG-1,wu01,469.0,293,2235.0,450,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte,AAACCTGCACCACGTG-1,wu01.AAACCTGCACCACGTG.1
wu01_AAACCTGTCTAACTTC-1,wu01,482.0,243,2436.0,400,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Malignant,In-house: EUS-FNB,wu01,Malignant_0,Malignant_0,Malignant_0,False,Malignant - Classical Low,AAACCTGTCTAACTTC-1,wu01.AAACCTGTCTAACTTC.1
wu01_AAACGGGCAAGGCTCC-1,wu01,1516.0,717,2944.0,717,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte,AAACGGGCAAGGCTCC-1,wu01.AAACGGGCAAGGCTCC.1
wu01_AAACGGGTCATTCACT-1,wu01,876.0,453,2491.0,524,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte,AAACGGGTCATTCACT-1,wu01.AAACGGGTCATTCACT.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCAAGTTGTCGT-1,G9903,6484.0,1875,9150.0,1886,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF,TTTGTCAAGTTGTCGT-1,G9903.TTTGTCAAGTTGTCGT.1
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCACAACTTGAC-1,G9903,8367.0,2286,9400.0,2286,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF,TTTGTCACAACTTGAC-1,G9903.TTTGTCACAACTTGAC.1
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCACATATGGTC-1,G9903,11476.0,3055,10387.0,3055,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Immune,Chan-Seng-Yue,G9903,immune,U1,immune,False,U1,TTTGTCACATATGGTC-1,G9903.TTTGTCACATATGGTC.1
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCATCCGAACGC-1,G9903,20314.0,3975,10684.0,3464,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,Malignant,Chan-Seng-Yue,G9903,Malignant_0,Malignant_0,Malignant_0,False,Malignant - Classical Low,TTTGTCATCCGAACGC-1,G9903.TTTGTCATCCGAACGC.1


In [86]:
list(adata.X[0].toarray().flatten())[:100]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.3862943611198906,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6931471805599453,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0986122886681096,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [88]:
list(adata.layers['counts'][0].toarray().flatten())[:100]

[0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 2.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 0.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 11.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0]

In [89]:
a = adata.copy()
a.obs = a.obs.set_index('cell_ids')
a.obs = a.obs[['sample_id', 'cell_type', 'final_cell_types', 'dataset']]
a

AnnData object with n_obs × n_vars = 151336 × 30840
    obs: 'sample_id', 'cell_type', 'final_cell_types', 'dataset'
    var: 'is_highly_variable'
    uns: 'cell_type_colors', 'cell_type_level_2_colors', 'cell_type_level_3_colors', 'dataset_colors', 'rank_genes_groups'
    obsm: 'X_umap'
    layers: 'counts'

In [90]:
sorted(set(a.obs['final_cell_types']))

['Acinar',
 'B cell',
 'CAF',
 'CD4 T cell',
 'CD4_CD8_T_cell - exhausted',
 'CD8 T cell',
 'DC',
 'Endothelial',
 'Erythrocyte',
 'Malignant - ADEX',
 'Malignant - Basal',
 'Malignant - Classical High',
 'Malignant - Classical Low',
 'Malignant - Mixed',
 'Mast',
 'Monocyte',
 'NK',
 'Plasma',
 'Stellate 1',
 'Stellate 2',
 'Treg',
 'U1']

In [91]:
# remove U1
a = a[a.obs['final_cell_types']!='U1']

In [92]:
m = {
    'CD4_CD8_T_cell - exhausted': 'CD8 T cell',
    'Malignant - ADEX': 'Malignant',
    'Malignant - Basal': 'Malignant',
    'Malignant - Classical High': 'Malignant',
    'Malignant - Classical Low': 'Malignant',
    'Malignant - Mixed': 'Malignant',
    'Stellate 1': 'Stellate',
    'Stellate 2': 'Stellate'
}
a.obs['renamed_cell_type'] = [m.get(x, x) for x in a.obs['final_cell_types']]
a.obs['renamed_cell_type'] = [x.replace(' ', '.') for x in a.obs['renamed_cell_type']]
set(a.obs['renamed_cell_type'])

Trying to set attribute `.obs` of view, copying.


{'Acinar',
 'B.cell',
 'CAF',
 'CD4.T.cell',
 'CD8.T.cell',
 'DC',
 'Endothelial',
 'Erythrocyte',
 'Malignant',
 'Mast',
 'Monocyte',
 'NK',
 'Plasma',
 'Stellate',
 'Treg'}

In [93]:
a.var.index = [x.split('.')[0] for x in a.var.index]
a.var

,is_highly_variable
AL669831,True
FAM87B,True
LINC00115,True
SAMD11,True
NOC2L,True
...,...
RP11-108P20,True
GS1-122H1,True
SLC26A4-AS1,True
RP11-403B2,True


In [94]:
pc = pd.read_csv('../data/protein_coding_genes/protein_coding_genes.txt', sep='\t')
pc

,gene,entrezgene_id,ensembl_gene_id,gene_biotype
0,A1BG,1.0,ENSG00000121410,protein_coding
1,A1BG-AS1,503538.0,ENSG00000268895,lncRNA
2,A1CF,29974.0,ENSG00000148584,protein_coding
3,A2M,2.0,ENSG00000175899,protein_coding
4,A2M-AS1,144571.0,ENSG00000245105,lncRNA
...,...,...,...,...
41941,ZYX,7791.0,ENSG00000159840,protein_coding
41942,ZYX,7791.0,ENSG00000285443,protein_coding
41943,ZYXP1,NaN,ENSG00000274572,unprocessed_pseudogene
41944,ZZEF1,23140.0,ENSG00000074755,protein_coding


In [95]:
coding = set(pc['gene'])
g2b = {g:b for g, b in zip(pc['gene'], pc['gene_biotype'])}
a.var['biotype'] = [g2b.get(g, 'unknown') for g in a.var.index]
a.var

,is_highly_variable,biotype
AL669831,True,unknown
FAM87B,True,lncRNA
LINC00115,True,lncRNA
SAMD11,True,protein_coding
NOC2L,True,protein_coding
...,...,...
RP11-108P20,True,unknown
GS1-122H1,True,unknown
SLC26A4-AS1,True,lncRNA
RP11-403B2,True,unknown


In [96]:
a = a[:, a.var['biotype']=='protein_coding']
a

View of AnnData object with n_obs × n_vars = 141982 × 17830
    obs: 'sample_id', 'cell_type', 'final_cell_types', 'dataset', 'renamed_cell_type'
    var: 'is_highly_variable', 'biotype'
    uns: 'cell_type_colors', 'cell_type_level_2_colors', 'cell_type_level_3_colors', 'dataset_colors', 'rank_genes_groups'
    obsm: 'X_umap'
    layers: 'counts'

In [98]:
from collections import Counter
Counter(a.obs['dataset']).most_common()

[('Peng', 41761),
 ('Chan-Seng-Yue', 32116),
 ('In-house: Surgical', 30764),
 ('In-house: EUS-FNB', 28707),
 ('Lin', 8634)]

In [99]:
# just eus to avoid batch effects
a = a[a.obs['dataset']=='In-house: EUS-FNB']
a.shape

(28707, 17830)

In [101]:
new = a.copy()
new.X = new.layers['counts']
sc.pp.normalize_total(new, target_sum=1e6)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [102]:
# exp = pd.DataFrame(data=a.X.toarray(), columns=a.var.index.to_list(), index=a.obs.index.to_list())
exp = pd.DataFrame(data=new.X.toarray(), columns=new.var.index.to_list(), index=new.obs.index.to_list())
exp

,SAMD11,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,AGRN,C1orf159,TNFRSF18,TNFRSF4,...,KRTAP3-3,STH,CDH7,DEFB132,RIMS4,SCGB2B2,SRY,CACNG2,MROH2B,KLK14
wu01.AAACCTGCACATAACC.1,0.0,103.993347,0.0,0.000000,0.000000,103.993347,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu01.AAACCTGCACCACGTG.1,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu01.AAACCTGTCTAACTTC.1,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu01.AAACGGGCAAGGCTCC.1,0.0,0.000000,0.0,0.000000,0.000000,693.962524,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu01.AAACGGGTCATTCACT.1,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wu32.TTTGGTTCATCACGAT.1,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,91.69265,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu32.TTTGGTTTCTCGCATC.1,0.0,45.593395,0.0,0.000000,91.186790,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu32.TTTGTCACACGGTGTC.1,0.0,27.544794,0.0,13.772397,27.544794,13.772397,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wu32.TTTGTCACAGATGGGT.1,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
exp = exp.transpose()
exp['gene'] = exp.index.to_list()
exp = exp.sort_values('gene')
exp = exp.groupby('gene').mean()
exp.index.name = 'Gene'
exp

,wu01.AAACCTGCACATAACC.1,wu01.AAACCTGCACCACGTG.1,wu01.AAACCTGTCTAACTTC.1,wu01.AAACGGGCAAGGCTCC.1,wu01.AAACGGGTCATTCACT.1,wu01.AAACGGGTCTGTGCAA.1,wu01.AAAGCAAAGCCACCTG.1,wu01.AAAGTAGAGAACTGTA.1,wu01.AAAGTAGGTCGGGTCT.1,wu01.AAATGCCTCCCAGGTG.1,...,wu32.TTTCCTCTCGTTGCCT.1,wu32.TTTGCGCAGTCTCCTC.1,wu32.TTTGCGCCAAGTACCT.1,wu32.TTTGGTTCAAGGTTCT.1,wu32.TTTGGTTCACTAGTAC.1,wu32.TTTGGTTCATCACGAT.1,wu32.TTTGGTTTCTCGCATC.1,wu32.TTTGTCACACGGTGTC.1,wu32.TTTGTCACAGATGGGT.1,wu32.TTTGTCATCACAAACC.1
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,51.996674,0.0,0.0,0.000000,0.0,258.531525,0.0,0.0,0.000000,59.580551,...,86.465294,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000
A1CF,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,73.937149,0.000000,0.000000,0.00000,91.69265,45.593395,82.634384,0.0,0.000000
A2M,51.996674,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,57.643528,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000
A2ML1,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000
A4GALT,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,193.986420,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000
ZYG11B,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,39.720367,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000
ZYX,311.980042,0.0,0.0,693.962524,0.0,0.000000,0.0,0.0,387.972839,158.881470,...,461.148224,49.291435,0.000000,0.000000,0.00000,0.00000,45.593395,13.772397,0.0,0.000000


In [104]:
a = new

In [ ]:
# ID, CellType and Sample

In [105]:
meta = a.obs[['renamed_cell_type', 'sample_id', 'dataset']]
meta['ID'] = meta.index.to_list()
meta.columns = ['CellType', 'Sample', 'Dataset', 'ID']
meta.index.name = 'Index'
meta

/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_pdac_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CellType,Sample,Dataset,ID
Index,,,,
wu01.AAACCTGCACATAACC.1,Monocyte,wu01,In-house: EUS-FNB,wu01.AAACCTGCACATAACC.1
wu01.AAACCTGCACCACGTG.1,Monocyte,wu01,In-house: EUS-FNB,wu01.AAACCTGCACCACGTG.1
wu01.AAACCTGTCTAACTTC.1,Malignant,wu01,In-house: EUS-FNB,wu01.AAACCTGTCTAACTTC.1
wu01.AAACGGGCAAGGCTCC.1,Monocyte,wu01,In-house: EUS-FNB,wu01.AAACGGGCAAGGCTCC.1
wu01.AAACGGGTCATTCACT.1,Monocyte,wu01,In-house: EUS-FNB,wu01.AAACGGGTCATTCACT.1
...,...,...,...,...
wu32.TTTGGTTCATCACGAT.1,Malignant,wu32,In-house: EUS-FNB,wu32.TTTGGTTCATCACGAT.1
wu32.TTTGGTTTCTCGCATC.1,Malignant,wu32,In-house: EUS-FNB,wu32.TTTGGTTTCTCGCATC.1
wu32.TTTGTCACACGGTGTC.1,Malignant,wu32,In-house: EUS-FNB,wu32.TTTGTCACACGGTGTC.1


In [106]:
for x in meta['ID']:
    if ' ' in x or '-' in x:
        print(x)
        break

In [107]:
# cant start with number
meta['ID'] = [f'X{x}' for x in meta['ID']]
meta.index = [f'X{x}' for x in meta.index]
exp.columns = [f'X{x}' for x in exp.columns]

/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_pdac_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [108]:
# cant start with number
meta['Sample'] = [f'X{x}'.replace(' ', '.').replace('-', '.') for x in meta['Sample']]
meta['Dataset'] = [x.replace(' ', '.').replace('-', '.').replace(':', '') for x in meta['Dataset']]
meta

/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_pdac_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_pdac_v2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,CellType,Sample,Dataset,ID
Xwu01.AAACCTGCACATAACC.1,Monocyte,Xwu01,In.house.EUS.FNB,Xwu01.AAACCTGCACATAACC.1
Xwu01.AAACCTGCACCACGTG.1,Monocyte,Xwu01,In.house.EUS.FNB,Xwu01.AAACCTGCACCACGTG.1
Xwu01.AAACCTGTCTAACTTC.1,Malignant,Xwu01,In.house.EUS.FNB,Xwu01.AAACCTGTCTAACTTC.1
Xwu01.AAACGGGCAAGGCTCC.1,Monocyte,Xwu01,In.house.EUS.FNB,Xwu01.AAACGGGCAAGGCTCC.1
Xwu01.AAACGGGTCATTCACT.1,Monocyte,Xwu01,In.house.EUS.FNB,Xwu01.AAACGGGTCATTCACT.1
...,...,...,...,...
Xwu32.TTTGGTTCATCACGAT.1,Malignant,Xwu32,In.house.EUS.FNB,Xwu32.TTTGGTTCATCACGAT.1
Xwu32.TTTGGTTTCTCGCATC.1,Malignant,Xwu32,In.house.EUS.FNB,Xwu32.TTTGGTTTCTCGCATC.1
Xwu32.TTTGTCACACGGTGTC.1,Malignant,Xwu32,In.house.EUS.FNB,Xwu32.TTTGTCACACGGTGTC.1
Xwu32.TTTGTCACAGATGGGT.1,Malignant,Xwu32,In.house.EUS.FNB,Xwu32.TTTGTCACAGATGGGT.1


In [109]:
set(meta['Sample'])

{'Xwu01',
 'Xwu02',
 'Xwu03',
 'Xwu04',
 'Xwu05',
 'Xwu07',
 'Xwu09',
 'Xwu14',
 'Xwu15',
 'Xwu16',
 'Xwu18',
 'Xwu22',
 'Xwu23',
 'Xwu24',
 'Xwu25',
 'Xwu26',
 'Xwu27',
 'Xwu28',
 'Xwu29',
 'Xwu30',
 'Xwu31',
 'Xwu32'}

In [110]:
set(meta['Dataset'])

{'In.house.EUS.FNB'}

In [112]:
exp.to_csv(f'../data/ecotyper/single_cell/eusfnb_exp_12152021.txt', sep='\t')

In [114]:
meta.to_csv(f'../data/ecotyper/single_cell/eusfnb_exp_12152021_annotations.txt', sep='\t')

In [113]:
a.write_h5ad('../data/combined/ecotyper_start_12152021_eusfnb.h5ad')

... storing 'renamed_cell_type' as categorical
... storing 'biotype' as categorical


In [115]:
meta

,CellType,Sample,Dataset,ID
Xwu01.AAACCTGCACATAACC.1,Monocyte,Xwu01,In.house.EUS.FNB,Xwu01.AAACCTGCACATAACC.1
Xwu01.AAACCTGCACCACGTG.1,Monocyte,Xwu01,In.house.EUS.FNB,Xwu01.AAACCTGCACCACGTG.1
Xwu01.AAACCTGTCTAACTTC.1,Malignant,Xwu01,In.house.EUS.FNB,Xwu01.AAACCTGTCTAACTTC.1
Xwu01.AAACGGGCAAGGCTCC.1,Monocyte,Xwu01,In.house.EUS.FNB,Xwu01.AAACGGGCAAGGCTCC.1
Xwu01.AAACGGGTCATTCACT.1,Monocyte,Xwu01,In.house.EUS.FNB,Xwu01.AAACGGGTCATTCACT.1
...,...,...,...,...
Xwu32.TTTGGTTCATCACGAT.1,Malignant,Xwu32,In.house.EUS.FNB,Xwu32.TTTGGTTCATCACGAT.1
Xwu32.TTTGGTTTCTCGCATC.1,Malignant,Xwu32,In.house.EUS.FNB,Xwu32.TTTGGTTTCTCGCATC.1
Xwu32.TTTGTCACACGGTGTC.1,Malignant,Xwu32,In.house.EUS.FNB,Xwu32.TTTGTCACACGGTGTC.1
Xwu32.TTTGTCACAGATGGGT.1,Malignant,Xwu32,In.house.EUS.FNB,Xwu32.TTTGTCACAGATGGGT.1


In [ ]:
## previous

In [51]:
adata = sc.read_h5ad('../data/combined/inhouse_combined_08262021.h5ad')
adata

AnnData object with n_obs × n_vars = 52848 × 25718
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.1', 'seurat_clusters', 'active.ident'
    obsm: 'X_umap'
    layers: 'counts'

In [57]:
adata.obs['dataset'] = ['in_house_eus_fnb' if x[:2]=='wu' else 'in_house_surgical'
                       for x in adata.obs.index]

In [58]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,nCount_SCT,nFeature_SCT,integrated_snn_res.1,seurat_clusters,active.ident,dataset
wu01_AAACCTGCACATAACC.1,wu01,11822.0,3295,14.016241,3735.0,1349,12,12,Monocyte,in_house_eus_fnb
wu01_AAACCTGCACCACGTG.1,wu01,469.0,293,5.970149,2464.0,525,10,10,Monocyte,in_house_eus_fnb
wu01_AAACCTGTCTAACTTC.1,wu01,482.0,243,0.000000,2653.0,468,22,22,Malignant,in_house_eus_fnb
wu01_AAACGGGCAAGGCTCC.1,wu01,1516.0,717,2.704485,3164.0,728,4,4,Monocyte,in_house_eus_fnb
wu01_AAACGGGTCATTCACT.1,wu01,876.0,453,7.648402,2712.0,554,12,12,Monocyte,in_house_eus_fnb
...,...,...,...,...,...,...,...,...,...,...
TWCE.HT081P1.XB1_TTTGTTGAGCGCCTTG.1,TWCE-HT081P1-XB1,424.0,284,5.188679,1005.0,314,13,13,Fibroblast,in_house_surgical
TWCE.HT081P1.XB1_TTTGTTGAGTAACCTC.1,TWCE-HT081P1-XB1,424.0,278,5.188679,999.0,311,13,13,Fibroblast,in_house_surgical
TWCE.HT081P1.XB1_TTTGTTGCAGTTGGTT.1,TWCE-HT081P1-XB1,276.0,207,1.811594,880.0,274,24,24,Monocyte,in_house_surgical
TWCE.HT081P1.XB1_TTTGTTGGTACTGCCG.1,TWCE-HT081P1-XB1,372.0,248,8.333333,938.0,308,17,17,Dendritic,in_house_surgical


In [52]:
adata.obs.index = [x.replace('-', '.') for x in adata.obs.index]
adata.var.index = [x.split('.')[0] for x in adata.var.index]
adata.var.index = [x.replace('-', '.') for x in adata.var.index]

In [53]:
len(adata.var.index), len(set(adata.var.index))

(25718, 24263)

In [54]:
def get_expression_matrix(a):
    counts = a.layers['counts']
    a.X = counts
    
    sc.pp.normalize_total(a, target_sum=1e6)
    
    exp = pd.DataFrame(data=a.X.toarray(), index=a.obs.index.to_list(),
                       columns=a.var.index.to_list())
    exp = exp.transpose()
    exp['Gene'] = exp.index.to_list()
    exp = exp.groupby('Gene').mean()
    
    return exp

In [59]:
# m = {
#     'Chan-Seng-Yue': 'chan_seng_yue',
#     'In-house: EUS-FNB': 'in_house_eus_fnb',
#     'In-house: Surgical': 'in_house_surgical',
#     'Lin': 'lin',
#     'Peng': 'peng'
# }

In [62]:
for dataset in sorted(set(adata.obs['dataset'])):
    print(dataset)
    exp = get_expression_matrix(adata[adata.obs['dataset']==dataset])
    exp.to_csv(f'../data/ecotyper/single_cell/{dataset}.txt', sep='\t')

in_house_eus_fnb


/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


in_house_surgical


/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/scipy/sparse/_index.py:116: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [63]:
for dataset in sorted(set(adata.obs['dataset'])):
    a = adata[adata.obs['dataset']==dataset]
    
    meta = a.obs[['active.ident']]
    meta.columns = ['CellType']
    meta.index.name = 'ID'
    meta.to_csv(f'../data/ecotyper/single_cell/{dataset}_annotations.txt', sep='\t')

/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [10]:
adata = sc.read_h5ad('../data/combined/ecotyper_sc_start_08232021.h5ad')
adata

AnnData object with n_obs × n_vars = 151336 × 30840
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'CELL', 'CONDITION', 'Patient', 'Type', 'Cell_type', 'integrated_snn_res.1.25', 'seurat_clusters', 'integrated_snn_res.2', 'integrated_snn_res.3', 'integrated_snn_res.1', 'integrated_snn_res.0.5', 'active.ident', 'cell_type', 'dataset', 'sample_id', 'cell_type_level_2', 'cell_type_level_3', 'tumor_original_seurat', 'malignant_exclude', 'final_cell_types'
    var: 'is_highly_variable'
    uns: 'cell_type_colors', 'cell_type_level_2_colors', 'cell_type_level_3_colors', 'dataset_colors', 'rank_genes_groups'
    obsm: 'X_umap'
    layers: 'counts'

In [11]:
adata.layers['counts']

<151336x30840 sparse matrix of type '<class 'numpy.float32'>'
	with 307110358 stored elements in Compressed Sparse Row format>

In [12]:
sorted(set(adata.obs['dataset']))

['Chan-Seng-Yue', 'In-house: EUS-FNB', 'In-house: Surgical', 'Lin', 'Peng']

In [13]:
sorted(set(adata.obs['cell_type']))

['Acinar',
 'B cell',
 'CAF',
 'DC',
 'Endothelial',
 'Erythrocyte',
 'Immune',
 'Malignant',
 'Mast',
 'Monocyte',
 'Plasma',
 'Stellate 1',
 'Stellate 2']

In [14]:
len(adata.obs.index), len(set(adata.obs.index))

(151336, 151336)

In [15]:
len(adata.var.index), len(set(adata.var.index))

(30840, 30840)

In [16]:
adata.var.index

Index(['AL669831.5', 'FAM87B', 'LINC00115', 'SAMD11', 'NOC2L', 'KLHL17',
       'PLEKHN1', 'HES4', 'ISG15', 'AGRN',
       ...
       'RP11-766N7.3', 'RP11-956E11.1', 'RP11-12K6.2', 'WASF3-AS1',
       'RP11-7F17.3', 'RP11-108P20.4', 'GS1-122H1.2', 'SLC26A4-AS1',
       'RP11-403B2.6', 'AP005530.1'],
      dtype='object', length=30840)

In [17]:
adata.var.index = [x.split('.')[0] for x in adata.var.index]
len(adata.var.index), len(set(adata.var.index))

(30840, 27949)

make R friendly

In [18]:
adata.obs.index = [x.replace('-', '.') for x in adata.obs.index]

In [19]:
adata.var.index = [x.replace('-', '_') for x in adata.var.index]

save each dataset as CPM

In [20]:
def get_expression_matrix(a):
    counts = a.layers['counts']
    a.X = counts
    
    sc.pp.normalize_total(a, target_sum=1e6)
    
    exp = pd.DataFrame(data=a.X.toarray(), index=a.obs.index.to_list(),
                       columns=a.var.index.to_list())
    exp = exp.transpose()
    exp['Gene'] = exp.index.to_list()
    exp = exp.groupby('Gene').mean()
    
    return exp

In [24]:
m = {
    'Chan-Seng-Yue': 'chan_seng_yue',
    'In-house: EUS-FNB': 'in_house_eus_fnb',
    'In-house: Surgical': 'in_house_surgical',
    'Lin': 'lin',
    'Peng': 'peng'
}

In [22]:
for dataset in sorted(set(adata.obs['dataset'])):
    print(dataset)
    exp = get_expression_matrix(adata[adata.obs['dataset']==dataset])
    d = m[dataset]
    exp.to_csv(f'../data/ecotyper/single_cell/{d}.txt', sep='\t')

Chan-Seng-Yue


/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/scipy/sparse/_index.py:116: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In-house: EUS-FNB


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In-house: Surgical


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Lin


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Peng


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [25]:
for dataset in sorted(set(adata.obs['dataset'])):
    a = adata[adata.obs['dataset']==dataset]
    d = m[dataset]
    
    z = {
        'CD8 T cell - exausted': 'CD4_CD8_T_cell_exhausted',
    }
    a.obs['cell_type_level_3'] = [z.get(x, x) for x in a.obs['cell_type_level_3']]
    a.obs['cell_type_level_3'] = [x.replace(' ', '_') for x in a.obs['cell_type_level_3']]
    
    meta = a.obs[['cell_type_level_3']]
    meta.columns = ['CellType']
    meta.index.name = 'ID'
    meta.to_csv(f'../data/ecotyper/single_cell/{d}_annotations.txt', sep='\t')

/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
Variabl

In [26]:
pd.read_csv('../data/ecotyper/single_cell/in_house_eus_fnb_annotations.txt', sep='\t')

,ID,CellType
0,wu01_AAACCTGCACATAACC.1,Monocyte
1,wu01_AAACCTGCACCACGTG.1,Monocyte
2,wu01_AAACCTGTCTAACTTC.1,Malignant_0
3,wu01_AAACGGGCAAGGCTCC.1,Monocyte
4,wu01_AAACGGGTCATTCACT.1,Monocyte
...,...,...
31767,wu32_TTTGGTTCATCACGAT.1,Malignant_1
31768,wu32_TTTGGTTTCTCGCATC.1,Malignant_3
31769,wu32_TTTGTCACACGGTGTC.1,Malignant_1
31770,wu32_TTTGTCACAGATGGGT.1,Malignant_3
